In [2]:
import numpy as np
import pandas as pd
import json
import os

In [3]:
def read_json_files(directory):
    files = os.listdir(directory)
    json_files = [file for file in files if file.endswith('.json')]

    dfs = []
    for file in json_files:
        file_path = os.path.join(directory, file)
        json_data = pd.read_json(file_path)
        dfs.append(json_data)

    dataframe = pd.concat(dfs, ignore_index=True)
    return dataframe

df = read_json_files('.')

df

,image,objects
0,"{'fileName': '20220622_SD04_7973.JPG', 'detail...","[{'bndbox': {'minX': 0.904863, 'minY': 0.90394..."
1,"{'fileName': '20220622_SD04_7892.JPG', 'detail...","[{'bndbox': {'minX': 0.560254, 'minY': 0.41549..."
2,"{'fileName': '20220622_SD04_7779.JPG', 'detail...","[{'bndbox': {'minX': 0.48625799999999997, 'min..."
3,"{'fileName': '20220622_SD04_7965.JPG', 'detail...","[{'bndbox': {'minX': 0.293869, 'minY': 0.54554..."
4,"{'fileName': '20220622_SD04_7904.JPG', 'detail...","[{'bndbox': {'minX': 0.47357299999999997, 'min..."
...,...,...
6746,"{'fileName': '20220627_SD04_8847.JPG', 'detail...","[{'bndbox': {'minX': 0.21301799999999999, 'min..."
6747,"{'fileName': '20220627_SD04_8804.JPG', 'detail...","[{'bndbox': {'minX': 0.5897439999999999, 'minY..."
6748,"{'fileName': '20220627_SD04_8595.JPG', 'detail...","[{'bndbox': {'minX': 0.545652, 'minY': 0.20220..."
6749,"{'fileName': '20220627_SD04_8768.JPG', 'detail...","[{'bndbox': {'minX': 0.268245, 'minY': 0.04438..."


In [16]:
image = df['image'].to_dict()
file_n = []
for i in range(0, len(image)):
    file_n.append(image[i]['fileName'])
    
file_orientation =[]
for i in range(0, len(image)):
    if image[i]['details']['width'] < image[i]['details']['height']:
        orientation_s = 'portait'
        file_orientation.append(orientation_s)
    else: 
        orientation_s = 'landscape'
        file_orientation.append(orientation_s)

jpg_df = pd.DataFrame({'file_name': file_n, 'orientation': file_orientation})
jpg_df = jpg_df.sort_values('file_name').reset_index(drop=True)
jpg_df = jpg_df.reset_index().rename(columns={'index': 'jpg_id'})
jpg_df

,jpg_id,file_name,orientation
0,0,20220308_SD02_2923.JPG,portait
1,1,20220308_SD02_2924.JPG,portait
2,2,20220308_SD02_2946.JPG,portait
3,3,20220308_SD02_2954.JPG,landscape
4,4,20220308_SD02_2955.JPG,landscape
...,...,...,...
6746,6746,20220825_SD05_6957.JPG,portait
6747,6747,20220825_SD05_6958.JPG,portait
6748,6748,20220825_SD05_6960.JPG,portait
6749,6749,20220825_SD05_6963.JPG,portait


In [5]:
obj = df['objects'].to_dict()
obj
obj[0][0]['bndbox']
obj[0][0]['category']['name']

'Toy-Grenade'

In [6]:
coord_list = []
bndbox_name_list = []
for i in range(len(obj)):
    coord = []
    bndbox_name = []
    for j in range(len(obj[i])):
        category_name = obj[i][j]['category']['name']
        bndbox_dict = obj[i][j]['bndbox']
        bndbox_tuple = (bndbox_dict['minX'], bndbox_dict['minY'], bndbox_dict['maxX'], bndbox_dict['maxY'])
        
        coord.append(bndbox_tuple)
        bndbox_name.append(category_name)
    coord_list.append(coord)
    bndbox_name_list.append(bndbox_name)

bndbox_df = pd.DataFrame({'bndbox_name': bndbox_name_list, 'coordinates': coord_list})
bndbox_df.head()


,bndbox_name,coordinates
0,"[Toy-Grenade, 48Z Face, 48Z Body]","[(0.904863, 0.903947, 1.0, 1.0), (0.4524309999..."
1,"[58K Face, 58K Body, 70G Face, 70G Body]","[(0.560254, 0.415498, 0.775899, 0.732673), (0...."
2,"[49Y Face, 120K Face, 120K Body, 49Y Body]","[(0.48625799999999997, 0.136385, 0.72093000000..."
3,"[42K Face, 42K Body, 68E Face, 68E Body]","[(0.293869, 0.54554, 0.46723000000000003, 0.78..."
4,"[48Z Face, 48Z Body]","[(0.47357299999999997, 0.044404, 0.638478, 0.2..."


In [7]:
bndbox_df['coordinates'][0]

[(0.904863, 0.903947, 1.0, 1.0),
 (0.45243099999999997, 0.104668, 0.6109939999999999, 0.35206299999999996),
 (0.281184, 0.10783899999999999, 0.697674, 1.0)]

In [17]:
coord_df = pd.concat([jpg_df['file_name'], bndbox_df], axis = 1)
coord_df.head(3)
coord_df = coord_df.explode(['bndbox_name', 'coordinates'])
coord_df = coord_df.sort_values('file_name')
coord_df.head(15)

,file_name,bndbox_name,coordinates
0,20220308_SD02_2923.JPG,Toy-Grenade,"(0.904863, 0.903947, 1.0, 1.0)"
0,20220308_SD02_2923.JPG,48Z Face,"(0.45243099999999997, 0.104668, 0.610993999999..."
0,20220308_SD02_2923.JPG,48Z Body,"(0.281184, 0.10783899999999999, 0.697674, 1.0)"
1,20220308_SD02_2924.JPG,58K Face,"(0.560254, 0.415498, 0.775899, 0.732673)"
1,20220308_SD02_2924.JPG,58K Body,"(0.49471499999999996, 0.421842, 0.775899, 1.0)"
1,20220308_SD02_2924.JPG,70G Face,"(0.346723, 0.0, 0.6363639999999999, 0.23788099..."
1,20220308_SD02_2924.JPG,70G Body,"(0.272727, 0.0, 0.807611, 1.0)"
2,20220308_SD02_2946.JPG,49Y Body,"(0.340381, 0.076122, 0.816068, 1.0)"
2,20220308_SD02_2946.JPG,120K Body,"(0.39957699999999996, 0.548712, 0.752643, 1.0)"
2,20220308_SD02_2946.JPG,120K Face,"(0.551797, 0.54554, 0.739958, 0.7834209999999999)"


In [9]:
coord_df.tail(5)

,file_name,bndbox_name,coordinates
117,20220825_SD05_6960.JPG,G942 Body,"(0.0, 0.173975, 0.8955559999999999, 0.876945)"
127,20220825_SD05_6963.JPG,G942 Face,"(0.581475, 0.164074, 0.846746, 0.424328)"
127,20220825_SD05_6963.JPG,G942 Body,"(0.0021219999999999998, 0.168317, 1.0, 0.929279)"
114,20220825_SD05_6967.JPG,G942 Face,"(0.22919399999999998, 0.011314999999999999, 0...."
114,20220825_SD05_6967.JPG,G942 Body,"(0.0042439999999999995, 0.015559, 0.965588, 1.0)"


In [14]:
coord_df.bndbox_name.unique()

array(['DF2I_Face', 'DF2I_Body', '7124_Body', '110E_Body', '7124_Face',
       '151J_Body', '151J_Face', '110E_Face', '67G_Face', '67G_Body',
       '94B_Body', '94B_Face', '72X_Face', '72X_Body', '143H_Body',
       '143H_Face', '69X_Face', '69X_Body', '87J_Body', '87J_Face',
       '14F_Face', '14F_Body', 'Toy-Unknown', '19J_Face', '19J_Body',
       '101G_Face', 'G701_Face', 'G701_Body', '68F_Body', '68F_Face',
       'Unknown Body', '151J Body', '151J Face', '110E Body', '110E Face',
       '7124 Face', '7124 Body', '49Y Face', '49Y Body', 'Unknown Face',
       '79G Face', '79G Body', '68E Face', '68E Body', '167I Face',
       '167I Body', '86I Face', '86I Body', '59E Face', '59E Body',
       '134J Body', '134J  Face', 'Toy-Dental Star', '35Y Body',
       '35Y Face', 'Toy-Dumbbell', 'G942 Face', 'G942 Body', '144H Body',
       '144H Face', '58I Face', '58I Body', '46J Body', '46J Face',
       '26J Body', '26J Face', 'DF2I Face', 'DF2I Body', '48Z Face',
       '48Z Body', '70

In [10]:
toy_coord = coord_df[coord_df['bndbox_name'].str.contains('toy', case=False)]
monkey_coord = coord_df[~coord_df['bndbox_name'].str.contains('toy', case=False)]

AttributeError: 'DataFrame' object has no attribute 'bndbox_name'

In [11]:
monkey_coord[['monkey_name', 'body_part']] = monkey_coord['bndbox_name'].str.split('_| ', 1, expand=True)
monkey_coord = monkey_coord.drop('bndbox_name',axis=1)
monkey_coord

/var/folders/59/wy2tkbm92h3chv3scjq0_q100000gp/T/ipykernel_71002/2707140005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monkey_coord[['monkey_name', 'body_part']] = monkey_coord['bndbox_name'].str.split('_| ', 1, expand=True)
/var/folders/59/wy2tkbm92h3chv3scjq0_q100000gp/T/ipykernel_71002/2707140005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monkey_coord[['monkey_name', 'body_part']] = monkey_coord['bndbox_name'].str.split('_| ', 1, expand=True)


,file_name,coordinates,monkey_name,body_part
5801,20220308_SD02_2923.JPG,"(0.36186799999999997, 0.0028209999999999997, 0...",DF2I,Face
5801,20220308_SD02_2923.JPG,"(0.0, 0.0, 0.954399, 1.0)",DF2I,Body
5800,20220308_SD02_2924.JPG,"(0.34282199999999996, 0.0, 0.8845649999999999,...",DF2I,Face
5800,20220308_SD02_2924.JPG,"(0.0, 0.0028209999999999997, 0.975560999999999...",DF2I,Body
2462,20220308_SD02_2946.JPG,"(0.002116, 0.0, 0.387262, 0.7334269999999999)",7124,Body
...,...,...,...,...
117,20220825_SD05_6960.JPG,"(0.0, 0.173975, 0.8955559999999999, 0.876945)",G942,Body
127,20220825_SD05_6963.JPG,"(0.581475, 0.164074, 0.846746, 0.424328)",G942,Face
127,20220825_SD05_6963.JPG,"(0.0021219999999999998, 0.168317, 1.0, 0.929279)",G942,Body
114,20220825_SD05_6967.JPG,"(0.22919399999999998, 0.011314999999999999, 0....",G942,Face


In [12]:
monkey_coord.body_part.unique()

array(['Face', 'Body', ' Face', '1 Body', '1 Face', '2 Body', 'Face ',
       'body', 'Body ', None, 'face'], dtype=object)

In [ ]:
 = coord_df[monkey_coord['body_part'].str.contains('face', case=False)]

In [21]:
toy_coord['bndbox_name'] = toy_coord['bndbox_name'].str.replace('Toy-', '')
toy_coord

/var/folders/59/wy2tkbm92h3chv3scjq0_q100000gp/T/ipykernel_70452/3485981414.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toy_coord['bndbox_name'] = toy_coord['bndbox_name'].str.replace('Toy-', '')


,file_name,bndbox_name,coordinates
1394,20220308_SD02_3022.JPG,Unknown,"(0.0, 0.895628, 0.16083, 1.0)"
1398,20220308_SD02_3029.JPG,Unknown,"(0.7477809999999999, 0.818182, 1.0, 0.97861000..."
1961,20220317_SD02_3125.JPG,Dental Star,"(0.6565219999999999, 0.727354, 0.947826, 0.825..."
2013,20220317_SD02_3165.JPG,Dumbbell,"(0.321212, 0.748602, 1.0, 1.0)"
1306,20220331_SD02_3342.JPG,Grenade,"(0.40405599999999997, 0.365112, 0.528861, 0.61..."
...,...,...,...
3754,20220810_SD05_6742.JPG,Challenger Ball,"(0.5826089999999999, 0.0, 1.0, 0.3202100000000..."
3709,20220810_SD05_6743.JPG,Challenger Ball,"(0.6869569999999999, 0.001449, 1.0, 0.22458199..."
430,20220825_SD05_6824.JPG,Kong,"(0.8576849999999999, 0.8910309999999999, 0.944..."
437,20220825_SD05_6825.JPG,Kong,"(0.8842500000000001, 0.928039, 0.963947, 1.0)"


In [11]:
#jpg_df.to_csv('../fromjson_photos.csv', index=False)
#coord_df.to_csv('../fromjson_coord.csv', index=False)
toy_coord.to_csv('../fromjson_toyCoord.csv', index = False)